# KERAS MODELLERİMİZİ TFLITE 'A DÖNÜŞTÜRME
## ÖRNEK BİR MODEL OLUŞTURALIM : MNIST VERİ SETİNİ MODELLEYELİM

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


11493376/11490434 [==============================] - 2s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 14s 225us/step - loss: 0.2720 - acc: 0.9167 - val_loss: 0.0578 - val_acc: 0.9819
Epoch 2/12
60000/60000 [==============================] - 9s 143us/step - loss: 0.0887 - acc: 0.9739 - val_loss: 0.0394 - val_acc: 0.9864
Epoch 3/12
60000/60000 [==============================] - 9s 143us/step - loss: 0.0675 - acc: 0.9790 - val_loss: 0.0355 - val_acc: 0.9885
Epoch 4/12
60000/60000 [==============================] - 9s 143us/step - loss: 0.0551 - acc: 0.9836 - val_loss: 0.0331 - val_acc: 0.9893
Epoch 5/12
60000/60000 [

## "keras_mdl.h5" ismiyle keras modelimizi kaydedelim

In [0]:
keras_model="keras_mdl.h5"
keras.models.save_model(model,keras_model)

In [29]:
!ls

keras_mdl.h5  sample_data


## Modelimizi dönüştürelim

In [31]:
from tensorflow.contrib import lite
donustur=lite.TocoConverter.from_keras_model_file(keras_model)
tflite_model=donustur.convert()
open('tflite_mld.tflite',"wb").write(tflite_model)

Instructions for updating:
Use `lite.TFLiteConverter.from_keras_model_file` instead.
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


4801492

## "tflite_mld.tflite" ismiyle modelimiz oluşmuş mu kontrol edelim

In [32]:
!ls

keras_mdl.h5  sample_data  tflite_mld.tflite


## COLAB ortamında oluşturduğumuz tflite modeli bilgisayarımıza indirelim

In [0]:
from google.colab import files
files.download('tflite_mld.tflite') 

# Faydalı olması dileğiyle